In [ ]:
# -*- coding: utf-8 -*-
"""
@author: domist07
"""
# Load packages
import sys
import gempy as gp
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from gempy.plot import visualization_2d_pro as vv
from gempy.plot import vista
# import pyvista as pv
from importlib import reload
reload(vista)

In [ ]:
# %% read data
path_sp = "322_Influence-surface-thickness_Points.csv"
df_sp = pd.read_csv(path_sp)

geo_data = gp.create_model('Issue322')
geo_data.set_surface_points(df_sp, add_basement=True)
geo_data.surface_points.df['smooth'] = 0

path_ori = "322_Influence-surface-thickness_Orientations.csv"
df_ori = pd.read_csv(path_ori)

geo_data.orientations.df = df_ori

In [ ]:
# %% Set surfaces
gp.map_series_to_surfaces(geo_data, {"Verw1": ('Verw1'), "Verw2": ('Verw2'),
                                     "Auffuel": ('Quartaer'),
                                     "Sand": ('verwKeuper', 'Sandstein'),
                                     "Schluff": ('Schluffstein')},
                          sort_geometric_data=True,
                          remove_unused_series=True)

geo_data.set_is_fault(['Verw1', 'Verw2'], change_color=False)
geo_data.series.set_bottom_relation("Auffuel", 'Onlap')
geo_data.faults.faults_relations_df.Auffuel = False
geo_data.faults.faults_relations_df.Sand = False

Min = np.array([np.min(geo_data.surface_points.df['X']),
                np.min(geo_data.surface_points.df['Y']),
                np.min(geo_data.surface_points.df['Z'])])
Max = np.array([np.max(geo_data.surface_points.df['X']),
                np.max(geo_data.surface_points.df['Y']),
                np.max(geo_data.surface_points.df['Z'])])

geo_data.grid.create_regular_grid([Min[0]-20, Max[0]+20,
                                   Min[1]-20, Max[1]+20,
                                   Min[2]-20, Max[2]+20],
                                  [50, 50, 50])

In [ ]:
# %% add Topography
geo_data.grid.create_topography(source='gdal',
    filepath="322_Influence-surface-thickness_GOK.csv")

# %% Set sections
section_dict = {'LS1': ([595, 274],
                        [610, 552],
                        [200, 200]),
                'LS2': ([645, 358],
                        [646, 549],
                        [200, 200]),
geo_data.grid.create_section_grid(section_dict)

In [ ]:
# %% Interpolation
gp.set_interpolator(geo_data, compile_theano=True,
                    theano_optimizer='fast_run', verbose=[])
# geo_data.modify_kriging_parameters('range', 100)
# geo_data.modify_kriging_parameters('$C_o$', 100)

gp.compute_model(geo_data, sort_surfaces=False, compute_mesh=True)

In [ ]:
# plot contacts
for i in geo_data.grid.sections.names:
    p = gp._plot.plot_2d(geo_data)
    sec_name = i
    a = p.add_section(sec_name)
    # p.plot_lith(a, sec_name)
    p.plot_topography(a, sec_name)
    p.plot_contacts(a, sec_name, a=0.5)
    p.create_figure()
del i, sec_name, a

In [ ]:
# plot 3D
gv = vista.Vista(geo_data, plotter_type='background', notebook=False,
                  real_time=True)
gv.plot_surfaces()
# gv.plot_surface_points()
# gv.plot_topography()
# gv.plot_orientations()